In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import pandas as pd
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [2]:
#CONVERT MNIST IMAGES FILES INTO TENSOR OF 4-DIMENSION(# OF IMAGES,HEIGHT,WIDTH, COLOR channels)
transform = transforms.ToTensor()

In [3]:
#Train Data
train_data = datasets.MNIST(root='.', train = True, download = True, transform = transform)


In [4]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: .
    Split: Train
    StandardTransform
Transform: ToTensor()

In [5]:
#Test Data
test_data = datasets.MNIST(root=',', train = False,  download = True, transform = transform)

In [6]:
ls

,/                                      Pytorch_ANN_MNIST_Practice.ipynb
FashionMNIST/                           Pytorch_Moores Law.ipynb
Linear Classification Note.ipynb        Untitled.ipynb
LinearClassification(withLogits).ipynb  moore.csv
MNIST/                                  mymodel.pt
Pytorch ANN MNIST.ipynb


In [7]:
cd MNIST

/Users/mikkychilaka/DeepLearning Pytorch/MNIST


In [8]:
ls

,/     MNIST/ raw/


In [9]:
#Create a small batch size for images... lets say 10
train_loader = DataLoader(train_data, batch_size=10, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 10, shuffle = False)

In [10]:
#Define our CNN model
#Describe convolution layer and what its doing (2 convolution layers)
conv1 = nn.Conv2d(1,6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)

In [11]:
#Grab 1 MNIST record/images
for i, (X_Train, y_train) in enumerate(train_data):
    break

In [12]:
X_Train

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [13]:
X_Train.shape

torch.Size([1, 28, 28])

In [14]:
x = X_Train.view(1,1,28,28)

In [15]:
#Perform our first convolution 
x = F.relu(conv1(x)) #Rectified Linear Unit for our activation funtion


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


In [16]:
# 1 is single image, 6 is the filter 
x.shape

torch.Size([1, 6, 26, 26])

In [17]:
#Pass through the pooling layer
x = F.max_pool2d(x, 2 ,2) #kernel of 2 and stride of 2

In [18]:
x.shape

torch.Size([1, 6, 13, 13])

In [19]:
#Do our second convolution layer
x = F.relu(conv2(x))

In [20]:
x.shape

torch.Size([1, 16, 11, 11])

In [21]:
#Pooling layer 
x = F.max_pool2d(x,2,2)

In [22]:
x.shape

torch.Size([1, 16, 5, 5])

In [23]:
#Model class 
class ConvolutionNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,3,1)
        self.conv2 = nn.Conv2d(6,16,3,1)
        # Fully connected layer 
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)

        def foward(self, X):
            X = F.relu(self.conv1(X))
            X = F.max_pool2d(X,2,2)
            #Second Pass
            X = F.relu(self.conv2(X))
            X = F.max_pool2d(X,2,2)

            #Re-View to flatten it out
            X = X.view(-1, 16*5*5) #negative so we can vary the batch size

            #Fully connected layers
            X = F.relu(self.fc1(X))
            X = F.relu(self.fc2(X))
            X = self.fc3()
            return F.log_softmax(X, dim=1)

In [24]:
#Create instance for the model
torch.manual_seed(41)
model = ConvolutionNetwork()
model

ConvolutionNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [25]:
#Loss Function optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #Smaller the learning rate longer it takes to train
 

In [26]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [27]:
import time
start_time = time.time()

#Create Variables to track things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

#For loops for epochs
for i in range(epochs):
    training_correct = 0
    testing_correct = 0

    #Train
    for b,(X_Train, y_train) in enumerate(train_loader):
        b+=1
        y_pred = model(X_Train) #get predicted values from training dataset
        loss = criterion(y_pred, y_train)
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        training_correct += batch_corr

    #update parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if b%600 == 0:
        print(f' Epoch{i} Batch{b} Loss{loss.item()}')

train_losses.append(loss)
train_correct.append(training_correct)
current_time = time.time()
total_time = current_time - start_time
print(f'Training took: {total/60} minutes!')

NotImplementedError: Module [ConvolutionNetwork] is missing the required "forward" function

In [ ]:
X_Train.shape

In [ ]:
y_train

In [ ]:
y_pred.data